[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sparks-baird/self-driving-lab-demo/blob/main/notebooks/6.1-multi-objective.ipynb)
# Multi-objective Optimization

In this notebook, we will use multi-objective optimization to find optimal trade-offs
between each of the 8 recorded wavelengths. This is in contrast to minimizing a
scalarized objective such as MAE, RMSE, or Frechet distance relative to a target
spectrum. We'll take a look to see which of MAE, RMSE, Frechet, and multi-objective
produce the most efficient searches.

One of the more sophisticated approaches to multi-objective optimization is the use of
the expected hypervolume improvement acquisition function. To understand this, you need
to understand what a Pareto front is.

> The Pareto front is defined as the set of non-dominated solutions, where each objective is considered as equally good.
> [Source](https://www.sciencedirect.com/topics/engineering/pareto-front#:~:text=The%20Pareto%20front%20is%20defined,Handbook%20of%20Neural%20Computation%2C%202017): Handbook of Neural Computation, 2017

The following figure illustrates the idea behind a Pareto front in two dimensions. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/Pareto_Efficient_Frontier_1024x1024.png/384px-Pareto_Efficient_Frontier_1024x1024.png)

> The red line is an example of a Pareto-efficient frontier, where the frontier and the area left and below it are a continuous set of choices. The red points on the frontier are examples of Pareto-optimal choices of production. Points off the frontier, such as N and K, are not Pareto-efficient, since there exist points on the frontier which Pareto-dominate them.

In other words, if you have two competing objectives, both of which you want to
maximize, the Pareto front (red line) is the set of optimal trade-offs between the two
objectives.

In materials science, one example of competing objectives is strength and ductility.
Typically, the higher the strength of a material, the lower the ductility. The Pareto
front is the set of optimal trade-offs between strength and ductility. Sometimes,
objectives are non-competing (i.e. they are correlated / increase with each other), in which case the Pareto front is
defined by a single point with the best possible value for each objective.

Here, we'll use continue to use the Ax platform. As before, we instantiate several
`SelfDrivingLabDemo`-s so that we can evaluate the average behavior across multiple repeats.

In [1]:
try:
    import google.colab
    %pip install self-driving-lab-demo
except:
    pass

In [66]:
from uuid import uuid4  # universally unique identifier
from self_driving_lab_demo import SelfDrivingLabDemo, mqtt_observe_sensor_data

pico_id = "test"  # @param {type:"string"}
num_repeats = 5   # @param {type:"integer"}
num_iter = 20 # @param {type:"integer"}
simulation = True # @param {type:"boolean"}
SESSION_ID = str(uuid4())  # random session ID
seeds = range(10, 10 + num_repeats)
print(f"session ID: {SESSION_ID}")

sdls = [
    SelfDrivingLabDemo(
        autoload=True,  # perform target data experiment automatically
        simulation=simulation,
        observe_sensor_data_fn=mqtt_observe_sensor_data,  # (default)
        observe_sensor_data_kwargs=dict(pico_id=pico_id, session_id=SESSION_ID),
        target_seed=seed,
    )
    for seed in seeds
]


session ID: 7ddeb5d8-17f3-43ae-b245-f765c315496e


In [67]:
sdls[0].evaluate(89, 89, 89)


{'ch410': 2050.0606978824158,
 'ch440': 44187.51171210927,
 'ch470': 198000.30797817995,
 'ch510': 71833.09757523212,
 'ch550': 21076.84622037978,
 'ch583': 3246.6032951750367,
 'ch620': 57421.50573479513,
 'ch670': 605.0644871207121,
 'mae': 14495.482507229324,
 'rmse': 23130.2337710172,
 'frechet': 52865.98532960104}

In [68]:
bo_results = []
channels = sdls[0].channel_names
channels


['ch410', 'ch440', 'ch470', 'ch510', 'ch550', 'ch583', 'ch620', 'ch670']

In [69]:
bounds = dict(R=sdls[0].bounds["R"], G=sdls[0].bounds["G"], B=sdls[0].bounds["B"])
params = [dict(name=nm, type="range", bounds=bnd) for nm, bnd in bounds.items()]
params


[{'name': 'R', 'type': 'range', 'bounds': [0, 89]},
 {'name': 'G', 'type': 'range', 'bounds': [0, 89]},
 {'name': 'B', 'type': 'range', 'bounds': [0, 89]}]

In [82]:
from ax.service.utils.instantiation import ObjectiveProperties

threshold = 1000.0 # or None to let the algorithm infer thresholds
if simulation:
    threshold = threshold * 5

obj_prefix = "delta_"
objectives = {
    obj_prefix + ch: ObjectiveProperties(minimize=True, threshold=threshold)
    for ch in channels
}
objectives


{'delta_ch410': ObjectiveProperties(minimize=True, threshold=5000.0),
 'delta_ch440': ObjectiveProperties(minimize=True, threshold=5000.0),
 'delta_ch470': ObjectiveProperties(minimize=True, threshold=5000.0),
 'delta_ch510': ObjectiveProperties(minimize=True, threshold=5000.0),
 'delta_ch550': ObjectiveProperties(minimize=True, threshold=5000.0),
 'delta_ch583': ObjectiveProperties(minimize=True, threshold=5000.0),
 'delta_ch620': ObjectiveProperties(minimize=True, threshold=5000.0),
 'delta_ch670': ObjectiveProperties(minimize=True, threshold=5000.0)}

In [83]:
# %%time
from ax.service.ax_client import AxClient

ax_clients = []

for sdl in sdls:
        
    ax_client = AxClient()
    ax_client.create_experiment(
        name="sdl-demo-moo",
        parameters=params,
        objectives=objectives,
        overwrite_existing_experiment=True,
    )

    def evaluate(parameters):
        R = parameters["R"]
        G = parameters["G"]
        B = parameters["B"]
        results = sdl.observe_sensor_data(R, G, B)
        delta_results = {
            obj_prefix + ch: abs(sdl.target_results[ch] - results[ch])
            for ch in channels
        }
        return delta_results

    for _ in range(num_iter):
        parameters, trial_index = ax_client.get_next_trial()
        ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))
    
    ax_clients.append(ax_client)


[INFO 10-14 10:29:14] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 10-14 10:29:14] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-14 10:29:14] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter G. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-14 10:29:14] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-14 10:29:14] ax.service.utils.instan

In [84]:
from sklearn.metrics import mean_absolute_error

maes = []
for sdl, ax_client in zip(sdls, ax_clients):
    pareto_result = ax_client.get_pareto_optimal_parameters(use_model_predictions=False)
    pareto_trial_nums = list(pareto_result.keys())
    target_inputs = sdl.get_target_inputs()
    pareto_optimal_colors = [
        (pareto_result[i][0]["R"], pareto_result[i][0]["G"], pareto_result[i][0]["B"])
        for i in pareto_trial_nums
    ]
    print(
        f"pareto optimal colors: {pareto_optimal_colors}\nTarget color: {target_inputs}"
    )
    mae = [mean_absolute_error(color, target_inputs) for color in pareto_optimal_colors]
    print(mae)
    maes.append(mae)


pareto optimal colors: [(79, 17, 74), (79, 21, 73)]
Target color: (85, 19, 74)
[2.6666666666666665, 3.0]
pareto optimal colors: [(16, 44, 54)]
Target color: (11, 45, 54)
[2.0]
pareto optimal colors: [(26, 80, 18), (25, 83, 15), (20, 79, 17)]
Target color: (22, 84, 17)
[3.0, 2.0, 2.3333333333333335]
pareto optimal colors: [(73, 74, 74), (79, 74, 71)]
Target color: (77, 76, 72)
[2.6666666666666665, 1.6666666666666667]
pareto optimal colors: [(74, 36, 65), (76, 32, 62), (80, 29, 65), (72, 30, 64)]
Target color: (74, 32, 63)
[2.0, 1.0, 3.6666666666666665, 1.6666666666666667]


Now let's compare with grid search using the three different scalarizers that are
implemented (MAE, RMSE, and Frechet distance).

In [75]:
from self_driving_lab_demo.utils.search import ax_bayesian_optimization

results = {}
for objective_name in ["mae", "rmse", "frechet"]:
    results[objective_name] = {}
    results[objective_name]["parameters"] = []
    results[objective_name]["color_mae"] = []
    for sdl in sdls:
        best_parameters, values, experiment, model = ax_bayesian_optimization(
            sdl, num_iter=num_iter, objective_name=objective_name
        )
        color = (best_parameters["R"], best_parameters["G"], best_parameters["B"])
        results[objective_name]["parameters"].append(color)
        target_color = sdl.get_target_inputs()
        results[objective_name]["color_mae"].append(
            mean_absolute_error(target_color, color)
        )


[INFO 10-14 10:01:54] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-14 10:01:54] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter G. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-14 10:01:54] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-14 10:01:54] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='R', parameter_type=INT, range=[0, 89]), RangeParameter(name='G', parameter_type=INT, range=[0, 89]), RangeParameter(name='B', parameter_type=INT, r

In [76]:
results["mae"]

{'parameters': [(89, 19, 74),
  (11, 44, 54),
  (22, 89, 17),
  (75, 76, 71),
  (75, 31, 63)],
 'color_mae': [1.3333333333333333,
  0.3333333333333333,
  1.6666666666666667,
  1.0,
  0.6666666666666666]}

In [77]:
results["rmse"]

{'parameters': [(88, 20, 74),
  (12, 45, 54),
  (23, 82, 18),
  (76, 75, 72),
  (74, 32, 63)],
 'color_mae': [1.3333333333333333,
  0.3333333333333333,
  1.3333333333333333,
  0.6666666666666666,
  0.0]}

In [78]:
results["frechet"]

{'parameters': [(85, 17, 74),
  (11, 44, 54),
  (24, 82, 16),
  (76, 77, 72),
  (78, 31, 62)],
 'color_mae': [0.6666666666666666,
  0.3333333333333333,
  1.6666666666666667,
  0.6666666666666666,
  2.0]}

In [101]:
import numpy as np
moo_maes = [np.mean(m) for m in maes]

In [102]:
import pandas as pd

df = pd.DataFrame(
    dict(
        moo=moo_maes,
        mae=results["mae"]["color_mae"],
        rmse=results["rmse"]["color_mae"],
        frechet=results["frechet"]["color_mae"],
    )
)
df

,moo,mae,rmse,frechet
0,2.833333,1.333333,1.333333,0.666667
1,2.000000,0.333333,0.333333,0.333333
2,2.444444,1.666667,1.333333,1.666667
3,2.166667,1.000000,0.666667,0.666667
4,2.083333,0.666667,0.000000,2.000000


In [103]:
df.mean()

moo        2.305556
mae        1.000000
rmse       0.733333
frechet    1.066667
dtype: float64

There are a few takeaways based on the results above (copied below for provenance).
```python
moo        2.305556
mae        1.000000
rmse       0.733333
frechet    1.066667
```
If we know exactly what we want and its easily quantified in a single objective (i.e.
minimize the MAE between the RGB values of two colors), then using a single, scalarized
objective is probably better. However, if we want to explore the space of possible solutions for competing
objectives (note that in this case, the objectives were correlated), then using a
multi-objective approach such as expected hypervolume improvement would still
be preferred. For example, if we had competing objectives of input power to
the LEDs and error relative to a target spectrum, and it wasn't immediately clear how to
trade off between the two objectives, then a multi-objective approach is still typically
preferred.

For more mathematical and theoretical details on a different task,
see the [Ax tutorial for multi-objective optimization](https://ax.dev/versions/0.2.0/tutorials/multiobjective_optimization.html).